In [1]:
import pandas as pd
import numpy as np
from allocations import Allocations
from metrics import Metrics

In [2]:
df = pd.read_csv("predictions/obermeyer/bootstrap_rf_0.csv")
data = pd.read_csv("data/obermeyer/obermeyer_data_cleaned.csv")

In [5]:
TEST_SIZE = 100
RASHOMON_EPSILON = 0.01
ITERATIONS_PER_SPLIT = 25

all_results = []
for SELECTION_RATE in [0.10, 0.25, 0.50]:
    for QUALIFICATION_COLUMN in ["qualified_gagne_3", "qualified_gagne_2", "qualified_gagne_1"]:
        a = Allocations(df, TEST_SIZE, SELECTION_RATE, QUALIFICATION_COLUMN, RASHOMON_EPSILON, ITERATIONS_PER_SPLIT)
        allocations = a.get_allocations()
        m = Metrics(allocations, data, QUALIFICATION_COLUMN)
        results = {}
        results["qualification_rate"] = QUALIFICATION_COLUMN
        results["selection_rate"] = SELECTION_RATE
        results["test_size"] = TEST_SIZE
        results["rashomon_epsilon"] = RASHOMON_EPSILON

        k_prime = m.k_prime()
        results["k_prime_avg"] = k_prime[0]
        results["k_prime_std"] = k_prime[1]
        
        n_prime = m.n_prime()
        results["n_prime_avg"] = n_prime[0]
        results["n_prime_std"] = n_prime[1]
        
        count_rashomon_allocations = m.count_rashomon_allocations()
        results["count_rashomon_allocations_avg"] = count_rashomon_allocations[0]
        results["count_rashomon_allocations_std"] = count_rashomon_allocations[1]
        
        count_rashomon_models = m.count_rashomon_models()
        results["count_rashomon_models_avg"] = count_rashomon_models[0]
        results["count_rashomon_models_std"] = count_rashomon_models[1]
        
        count_rashomon_models_per_allocation = m.count_rashomon_models_per_allocation()
        results["count_rashomon_models_per_allocation_avg"] = count_rashomon_models_per_allocation[0]
        results["count_rashomon_models_per_allocation_std"] = count_rashomon_models_per_allocation[1]
        
        systemic_exclusion = m.systemic_exclusion()
        results["systemic_exclusion_avg"] = systemic_exclusion[0]
        results["systemic_exclusion_std"] = systemic_exclusion[1]

        systemic_exclusion = m.systemic_exclusion_pairwise()
        results["systemic_exclusion_pairwise_avg"] = systemic_exclusion[0]
        results["systemic_exclusion_pairwise_std"] = systemic_exclusion[1]

        selections_by_qualification = m.selections_by_qualification()
        results["qualified_selections_avg"] = selections_by_qualification[0]
        results["qualified_selections_std"] = selections_by_qualification[1]
        results["unqualified_selections_avg"] = selections_by_qualification[2]
        results["unqualified_selections_std"] = selections_by_qualification[3]
        
        minority_selection_rate = m.minority_selection_rate("race", 1)
        results["minority_selection_avg"] = minority_selection_rate[0]
        results["minority_selection_std"] = minority_selection_rate[1]
        results["minority_selection_best_found"] = minority_selection_rate[2]
        
        best_minority_selection_rate = m.best_minority_selection_rate("race", 1)
        results["best_minority_selection_avg"] = best_minority_selection_rate[0]
        results["best_minority_selection_std"] = best_minority_selection_rate[1]
        
        feature_ratio_by_group = m.feature_ratio_by_group("race", 1, 0, "gagne_sum_t")
        results["feature_ratio_avg"] = feature_ratio_by_group[0]
        results["feature_ratio_std"] = feature_ratio_by_group[1]
        results["feature_ratio_best_found"] = feature_ratio_by_group[2]
        
        best_feature_ratio = m.best_feature_ratio_by_group("race", 1, 0, "gagne_sum_t")
        results["best_feature_ratio_avg"] = best_feature_ratio[0]
        results["best_feature_ratio_std"] = best_feature_ratio[1]
        all_results.append(results)
        
        h = m.local_homogenization()
        results["local_homog_avg"] = h[0]
        results["local_homog_std"] = h[1]
        results["baseline_local_homog_avg"] = h[2]
        results["baseline_local_homog_std"] = h[3]
        print(h)
        
        h = m.global_homogenization("age")
        results["global_homog_avg"] = h[0]
        results["global_homog_std"] = h[1]
        results["baseline_global_homog_avg"] = h[2]
        results["baseline_global_homog_std"] = h[3]
        print(h)
        
        all_results.append(results)

/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:70: RuntimeWarning: Mean of empty slice
  metric.append(np.nanmean(metric_inner))
/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:251: RuntimeWarning: Mean of empty slice
  metric_flattened = [np.nanmean(inner) for inner in metric]


(np.float64(0.034026715821016726), np.float64(0.024519142158864055), np.float64(0.25282858574809075), np.float64(0.03143789219580371))
(np.float64(1.4885559623567945), np.float64(0.142995510901804), np.float64(1.7328587801568505), np.float64(0.057181692666898896))


/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:70: RuntimeWarning: Mean of empty slice
  metric.append(np.nanmean(metric_inner))
/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:251: RuntimeWarning: Mean of empty slice
  metric_flattened = [np.nanmean(inner) for inner in metric]


(np.float64(0.028609268170640228), np.float64(0.026253438953010334), np.float64(0.2632306242748745), np.float64(0.030238170413159144))
(np.float64(1.5001114353601352), np.float64(0.16083126948329515), np.float64(1.7027228415761055), np.float64(0.05432615968271526))


/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:70: RuntimeWarning: Mean of empty slice
  metric.append(np.nanmean(metric_inner))
/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:251: RuntimeWarning: Mean of empty slice
  metric_flattened = [np.nanmean(inner) for inner in metric]


...

(np.float64(0.0731089802103086), np.float64(0.0657638599820458), np.float64(0.5921273972914816), np.float64(0.03631707484731803))
(np.float64(1.7632856335184266), np.float64(0.05493866630315471), np.float64(1.7328587801568505), np.float64(0.057181692666898896))


/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:70: RuntimeWarning: Mean of empty slice
  metric.append(np.nanmean(metric_inner))


(np.float64(0.04373449578256796), np.float64(0.05255837711093128), np.float64(0.5798846777616624), np.float64(0.04252343590639222))
(np.float64(1.7580732454782082), np.float64(0.06288406614002402), np.float64(1.7027228415761055), np.float64(0.05432615968271526))


/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:70: RuntimeWarning: Mean of empty slice
  metric.append(np.nanmean(metric_inner))


(np.float64(0.02584392322170881), np.float64(0.05477100538956645), np.float64(0.5914005623743094), np.float64(0.025428862349502167))
(np.float64(1.7523596160687884), np.float64(0.06522734508367664), np.float64(1.6928147599802896), np.float64(0.050276772431695))


In [6]:
all_results = pd.DataFrame(all_results)
all_results.to_csv("results.csv", index=False)